<a href="https://colab.research.google.com/github/Series-Parallel/Deo-Pathak-wasserstoff-AiInternTask/blob/main/Personal_Email_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
from io import StringIO

In [3]:
# ngrok chaning url
BASE_SERVER_URL = "https://ac3a-103-240-207-60.ngrok-free.app"

In [4]:
url = f"{BASE_SERVER_URL}/emails"

In [5]:
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

In [6]:
data.head()

,id,message_id,subject,sender,body,timestamp
0,1387,19611fbe67108ec1,No Subject,No Subject,"Just testing if my function is working or not,...",2025-04-08 02:33:39.631405
1,1386,1961201429fdcf52,No Subject,No Subject,"Hi,\n\nYour meeting has been confirmed for 202...",2025-04-08 02:33:39.151274
2,991,196115d9331bcbb0,No Subject,No Subject,This is AI sending u email by itself :P.,2025-04-08 00:18:49.728056
3,990,196117975eef36ad,Get Unlimited Prep for Your MBA Journey,Manhattan Prep <info@manhattanprep.com>,http://cloud.e-mail.manhattanprep.com/unsubscr...,2025-04-08 00:18:49.204786
4,699,1960e8c5bb48c839,You’re Using ChatGPT Wrong! Here’s How to Be A...,Medium <noreply@medium.com>,Stories for Pathakdeo\r\n@pathakdeo24 (https:/...,2025-04-07 23:07:37.358909


In [7]:
# data = pd.read_csv('emails.csv')

In [8]:
# data.head()

In [9]:
def create_text_representation(row):
    return f"""
    Sender: {row['sender']}
    Subject: {row['subject']}
    Date: {row['timestamp']}

    Email Body:
    {row['body']}
    """


In [10]:
!pip install chromadb

In [11]:
!pip install langchain_community

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import chromadb

In [13]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-13-86f3ffb55a21>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [14]:
data['text_representation'] = data.apply(create_text_representation, axis=1)

In [15]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [16]:
collection = chroma_client.get_or_create_collection(name="emails")

In [17]:
text = data['text_representation'].tolist()

In [18]:
ids = [str(index) for index in data.index]

In [19]:
metadatas = [
    {
        "sender": row["sender"],
        "subject": row["subject"],
        "timestamp": row["timestamp"]
    }
    for _, row in data.iterrows()
]

In [20]:
collection.add(
    ids=ids,
    documents=text,
    metadatas=metadatas
)

# LLM with RAG

## Installations and imports

In [21]:
!pip install langchain-mistralai

In [22]:
!pip install langchain-google-community

In [23]:
from langchain_mistralai.chat_models import ChatMistralAI

In [24]:
from langchain.vectorstores import Chroma

In [25]:
from google.colab import userdata

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [28]:
from langchain_google_community import GoogleSearchAPIWrapper

In [29]:
import json
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [30]:
import re
from datetime import datetime, timedelta

In [31]:
import time

## Building the LLM with RAG



In [32]:
# Create the vector store
db = Chroma(client=chroma_client, embedding_function=embeddings_model, collection_name="emails")

<ipython-input-32-e86f8ed11573>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(client=chroma_client, embedding_function=embeddings_model, collection_name="emails")


In [33]:
# Define a retriever interface
retriever = db.as_retriever()

In [34]:
api_key = userdata.get('MISTRAL_API_KEY')

In [35]:
# Define LLM

model = ChatMistralAI(mistral_api_key=api_key)

In [36]:
# Define prompt template

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [37]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [38]:
google_search = GoogleSearchAPIWrapper(
    google_api_key=userdata.get("GOOGLE_API_KEY"),
    google_cse_id=userdata.get("GOOGLE_CSE_ID")
)

In [39]:
last_llm_response_time = 0
COOLDOWN_SECONDS = 5

## LLM response filtering functions!

In [40]:
def draft_and_send_slack_message(user_query):
    """
    Uses the LLM to draft a Slack message and sends it via the Flask Slack API.
    """
    draft_slack_prompt = ChatPromptTemplate.from_template(
        """
        Draft a professional Slack message based on the user's request.

        Request: {user_query}

        Slack Message:
        """
    )

    formatted_prompt = draft_slack_prompt.format(user_query=user_query)
    result = model.invoke(formatted_prompt)

    slack_message = result.content.strip()

    try:
        response = requests.post(
            f"{BASE_SERVER_URL}/send_slack_message",
            json={"message": slack_message}
        )
        if response.status_code == 200:
            return "Slack message sent successfully!"
        else:
            return f"Slack API error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error calling Slack API: {str(e)}"

In [41]:
def draft_and_send_email(user_query):
    """
    Drafts a professional email using LLM and sends it via API.
    """
    draft_email_prompt = ChatPromptTemplate.from_template(
        """
        Draft a professional email based on the user's request.
        Include subject, recipient (if mentioned), and body.

        Request: {user_query}

        Email (format as JSON with 'to', 'subject', 'body'):
        """
    )

    formatted_prompt = draft_email_prompt.format(user_query=user_query)
    result = model.invoke(formatted_prompt)

    try:
        email_json = json.loads(result.content)
    except Exception as e:
        return f"Error parsing drafted email: {e}"

    response = requests.post(
        f"{BASE_SERVER_URL}/send_email",
        json=email_json
    )

    return f"Email sent! Server response: {response.text}"


In [42]:
def rag_query_with_web_search(user_query):
    """
    Handles user query with RAG. If it's about Slack, it sends via API. Otherwise answers from email context or falls back to web.
    """

    global last_llm_response_time

    now = time.time()
    if now - last_llm_response_time < COOLDOWN_SECONDS:
        wait_time = round(COOLDOWN_SECONDS - (now - last_llm_response_time), 1)
        return f"Please wait {wait_time} more second(s) before asking again to avoid rate limits."

    last_llm_response_time = now

    # making slack message
    if user_query.lower().startswith("draft a message on slack") or user_query.lower().startswith("draft a slack message"):
        return draft_and_send_slack_message(user_query)

    # making email
    if user_query.lower().startswith("draft an email"):
        return draft_and_send_email(user_query)

    response = retrieval_chain.invoke({"input": user_query})
    answer = response.get("answer", "").strip()

    needs_web_search = (
        not answer
        or answer.lower().startswith("the email does not provide enough context to determine")
        or answer.lower().startswith("the context does not provide information")
        or answer.lower().startswith("the context provided does not contain")
        or answer.lower().startswith("the context provided does not specify")
        or answer.lower().startswith("the provided context does not contain")
        or "does not provide enough context" in answer.lower()
        or "i don't know" in answer.lower()
        or "no relevant information" in answer.lower()
    )

    if needs_web_search:
        print("Couldn't find in emails. Hence, searching the Web...")
        search_result = google_search.run(user_query)

        web_prompt = ChatPromptTemplate.from_template(
            """
            Use the following web result context to answer the user's question.

            Web result:
            <context>
            {search_result}
            </context>

            Question: {user_query}

            Answer:
            """
        )

        web_chain = web_prompt | model
        result = web_chain.invoke({"search_result": search_result, "user_query": user_query})
        answer = result.content if hasattr(result, "content") else str(result)

    return answer

In [43]:
def check_and_notify_slack(llm_response):
    """
    Detects urgency from LLM response and sends important info to Slack via API.
    """
    pattern = r"\byes\b.*\b(urgent|important)\b.*\b(email|meeting)s?\b"

    if re.search(pattern, llm_response, re.IGNORECASE):
        # print("Urgent message detected in LLM response.")

        subject_patterns = [
            r"subject\s*:\s*['\"]?(.*?)(?=['\"\n\r]|$)",
            r"subject\s+is\s+['\"]?(.*?)(?=['\"\n\r]|$)",
            r"['\"]?(URGENT[^'\"]*)['\"]?"
        ]

        subject = None
        for pat in subject_patterns:
            match = re.search(pat, llm_response, re.IGNORECASE)
            if match:
                subject = match.group(1).strip()
                break

        if not subject:
            subject = "An important message needs attention."

        slack_message = f"From Head Office - {subject}"

        # Send to your Flask API
        try:
            response = requests.post(
                f"{BASE_SERVER_URL}/send_slack_message",
                json={"message": slack_message}
            )
            if response.status_code == 200:
                print("Slack message sent successfully.")
                return True
            else:
                print("Slack API responded with:", response.status_code, response.text)
        except Exception as e:
            print("Error while calling Slack API:", str(e))

    return False

In [44]:
def detect_meeting_intent(text):
    """
    Detects if there are any meetings in the response.
    """

    subject_match = re.search(r'subject\s+"([^"]+)"', text, re.IGNORECASE)
    summary = subject_match.group(1) if subject_match else "Meeting"

    pattern = re.compile(
        r"(?P<month>\w+)\s+(?P<day>\d{1,2})(?:st|nd|rd|th)?\s+at\s+"
        r"(?P<start_hour>\d{1,2}:\d{2})\s*(?P<start_ampm>AM|PM)"
        r"(?:\s*to\s*(?P<end_hour>\d{1,2}:\d{2})\s*(?P<end_ampm>AM|PM))?",
        re.IGNORECASE
    )

    match = pattern.search(text)
    if not match:
        return None

    data = match.groupdict()

    # Use static year for now
    date_str = f"{data['month']} {data['day']} 2025"
    try:
        date = datetime.strptime(date_str, "%B %d %Y")
    except Exception as e:
        return None

    # Start time
    start_time_str = f"{data['start_hour']} {data['start_ampm']}"
    start_time = datetime.strptime(start_time_str, "%I:%M %p")
    start = date.replace(hour=start_time.hour, minute=start_time.minute)

    # End time (if exists)
    if data["end_hour"] and data["end_ampm"]:
        end_time_str = f"{data['end_hour']} {data['end_ampm']}"
        end_time = datetime.strptime(end_time_str, "%I:%M %p")
        end = date.replace(hour=end_time.hour, minute=end_time.minute)
    else:
        end = start + timedelta(minutes=30)

    event = {
        "summary": summary,
        "start": start.strftime("%Y-%m-%dT%H:%M:%S"),
        "end": end.strftime("%Y-%m-%dT%H:%M:%S")
    }

    return event

In [45]:
def create_calendar_event(event_details):

  """
  It schedules meetings on the google calendar using its api and credentials.
  For this to work, credentials should be uploaded to colab
  """

  with open("/content/calendar_credentials.json") as f:
      service_account_info = json.load(f)

  creds = service_account.Credentials.from_service_account_info(
      service_account_info,
      scopes = ['https://www.googleapis.com/auth/calendar']
  )
  service = build('calendar','v3', credentials=creds)

  event = {
      "summary":event_details["summary"],
      "start":{
          "dateTime":event_details["start"],
          "timeZone":"Asia/Kolkata"
      },
      "end":{
          "dateTime":event_details["end"],
          "timeZone":"Asia/Kolkata"
      }
  }

  result = service.events().insert(calendarId="pathakdeo24@gmail.com", body=event).execute()
  return result.get("htmlLink")


In [46]:
def send_confirmation_email(recipient_email, subject, body):
  """
  This will send confirmation email to the sender who wants to schedule meetings
  """

  try:
        response = requests.post(
            f"{BASE_SERVER_URL}/send_email",
            json={
                "to": recipient_email,
                "subject": subject,
                "message": body
            }
        )
        response.raise_for_status()
        print("Confirmation email sent!")
  except Exception as e:
        print(f"Failed to send confirmation email: {e}")

In [47]:
def handle_query(user_query):
  answer = rag_query_with_web_search(user_query)

  check_and_notify_slack(answer)

  meeting = detect_meeting_intent(answer)
  if meeting:
      link = create_calendar_event(meeting)
      answer += f"\n\nMeeting added to your calendar: [View Event]({link})"

      match = re.search(r"[a-zA-Z0-9._%+-]+@gmail\.com", answer)
      recipient = match.group(0) if match else "pathakdeo11@gmail.com"

      subject = f"Meeting Confirmation: {meeting['summary']}"
      message = f"Hi,\n\nYour meeting has been confirmed for {meeting['start']}.\n\nRegards,\nAI Assistant"
      send_confirmation_email(recipient, subject, message)

  return answer

# Setting up Flask to interact with Frontend

## Installations


In [48]:
!pip install flask_ngrok

In [49]:
!pip install flask pyngrok

In [50]:
!pip install flask-cors

In [51]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from flask_cors import CORS

In [52]:
conf.get_default().auth_token =  userdata.get("NGROK_TOKEN")

## Server

In [53]:
app = Flask(__name__)
CORS(app)

@app.route('/query', methods=['POST'])
def query_llm():
    data = request.get_json()
    question = data.get('question', '')
    response = handle_query(question)
    return jsonify({'answer': response})

public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

app.run(port=5000)

Public URL: NgrokTunnel: "https://5dd0-35-196-68-55.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:11:19] "OPTIONS /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:11:21] "POST /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:12:16] "OPTIONS /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:12:17] "POST /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:12:51] "OPTIONS /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:12:52] "POST /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:14:21] "OPTIONS /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:14:22] "POST /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:15:10] "OPTIONS /query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 21:15:14] "POST /que

Slack message sent successfully.


# Testing and scratch commands

In [54]:
# important_subject = "Urgent: Payment failed for client XYZ"
# send_slack_message(important_subject)

In [55]:
# response = retrieval_chain.invoke({"input": "Is there any upcoming meetings?"})
# print(response["answer"])

In [56]:
# response56 = rag_query_with_web_search("Draft a message on slack saying - urgent meeting with Perfect Solution Company ")
# response56

In [57]:
# response2 = rag_query_with_web_search("Which country requires gre exam for entrance?")
# print(response2)

In [58]:
# url = "https://6469-103-240-207-60.ngrok-free.app/send_email"
# payload = {
#     "to": "pathakdeo11@gmail.com",
#     "subject": "Test from Colab",
#     "message": "Hello! Just testing the POST request."
# }

# response = requests.post(url, json=payload)
# print(response.status_code)
# print(response.text)

In [59]:
# response4 = handle_query("what day is today?")
# response4

In [60]:
# events = detect_meeting_intent("""A follow-up meeting regarding Q2 targets requested by someone named Anita from the email with subject "Follow-up Meeting Regarding Q2 Targets". The suggested date and time for this meeting is Friday, April 24th at 4:30 PM to 6:30 PM, but it is expected to only take about 30 minutes.""")
# print(events)

In [61]:
# response42 = handle_query("is there any email with subject - URGENT: Account issue  - followed by some other words i dont remeber?")
# response42

In [62]:
# response3 = handle_query("Can u add upcoming meetings to calendar?")
# print(response3)

In [63]:
# create_calendar_event(events)

In [64]:
# response241 = send_confirmation_email('pathakdeo11@gmail.com','Checking if function works!', 'Just testing if my function is working or not, if the email came into your mailbox its a good sign then! :)')
# response241